## Setup

### Imports

In [1]:
from typing import List
from pathlib import Path
from pprint import pprint
import re
import sys

import pandas as pd
import xlwings as xw

from buffered_iterator import BufferedIterator
import text_reader as tp
from sections import Rule, RuleSet, SectionBreak, ProcessingMethods, Section

### Logging

In [2]:
import logging
logging.basicConfig(format='%(name)-20s - %(levelname)s: %(message)s')
#logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger('Two Line SubSection Tests')
#logger.setLevel(logging.DEBUG)
logger.setLevel(logging.INFO)

### Display Functions

In [3]:
#%%writefile display_functions.py

In [4]:
# %% Function to compare context for two sections.
def compare_context(section1, section2):
    ctx_template = '{key:16s}:\t{item1:16s}\t{item2:16s}'
    context_1 = section1.context
    context_2 = section2.context
    keys_1 = set(context_1.keys())
    keys_2 = set(context_2.keys())
    all_keys = keys_1 | keys_2
    for key in all_keys:
        item1 = context_1.get(key, '')
        item2 = context_2.get(key, '')
        ctx_str = ctx_template.format(key=str(key), item1=str(item1), item2=str(item2))
        print(ctx_str)
        
        
# %% Compare Buffered Iterator contents
def buffered_iterator_compare(iter1, iter2=None, iter3=None, 
                              label1='From Iterator', 
                              label2='To Iterator', label3=''):
    
    def extract_attrs(buf_obj, requested_item, as_list=True):
        if not buf_obj:
            text = ''
        elif as_list:
            text = str(list(buf_obj.__getattribute__(requested_item)))
        else:
            text = str(buf_obj.__getattribute__(requested_item))
        return text
        
    def extract_attr_text(requested_item, iter1, iter2=None, iter3=None, 
                        as_list=True):    
        attr_text = {
            1: extract_attrs(iter1, requested_item, as_list),
            2: extract_attrs(iter2, requested_item, as_list),
            3: extract_attrs(iter3, requested_item, as_list),
        }
        return attr_text


    row_template = ''.join([
        '\t{Label:<20s}',
        '{first_iter_item:<35s}',
        '{second_iter_item:<35s}',
        '{third_iter_item:<35s}\n'
        ])   
    attr_group = {
        'Previous Items': ('previous_items', True),
        'Future Items': ('future_items', True),
        'Item Count': ('item_count', False),
        'Step Back': ('_step_back', False),
        'Buffer Size': ('buffer_size', False)
        }

    row_list = [
        row_template.format(
            Label='',
            first_iter_item=label1, 
            second_iter_item=label2, 
            third_iter_item=label3)
                ]

    for label, attr_s in attr_group.items():
        requested_item, as_list = attr_s
        text_group = extract_attr_text(requested_item, iter1, iter2, iter3, as_list)
        text_line = row_template.format(Label=label, 
                        first_iter_item=text_group[1],
                        second_iter_item=text_group[2],
                        third_iter_item=text_group[3])
        row_list.append(text_line)
    
    iterator_compare_str = ''.join(row_list)
    
    return iterator_compare_str

**********

# Section Break options

## Three line sections

In [5]:
GENERIC_TEST_TEXT2 = [
    'Text to be ignored',
    'StartSection A',
    'MiddleSection A',
    'EndSection A',
    'Unwanted text between sections',
    'StartSection B',
    'MiddleSection B',
    'EndSection B',
    'StartSection C',
    'MiddleSection C',
    'EndSection C',
    'Even more text to be ignored', 
    ]

**********

### Initial Section and Sub-Section Definitions

- Only definition line is:<br>
`processor=[sub_section]`

In [6]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )

full_section = Section(
    section_name='Full',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    #end_on_first_item=True
    processor=sub_section  
    )
pprint(full_section.read(GENERIC_TEST_TEXT2))

[['Text to be ignored',
  'StartSection A',
  'MiddleSection A',
  'EndSection A',
  'Unwanted text between sections',
  'StartSection B',
  'MiddleSection B',
  'EndSection B',
  'StartSection C',
  'MiddleSection C',
  'EndSection C',
  'Even more text to be ignored']]


- ![Good](../examples/Valid.png) Results in all lines in one sub-list

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      ['Text to be ignored',<br>
       'StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A',<br>
       'Unwanted text between sections',<br>
       'StartSection B',<br>
       'MiddleSection B',<br>
       'EndSection B',<br>
       'StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C',<br>
       'Even more text to be ignored']<br>
      ]</code></td>
  <td><code>
    [<br>
      ['Text to be ignored',<br>
       'StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A',<br>
       'Unwanted text between sections',<br>
       'StartSection B',<br>
       'MiddleSection B',<br>
       'EndSection B',<br>
       'StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C',<br>
       'Even more text to be ignored']<br>
      ]</code></td>
</table>



### Add start to Section Definition

- Section start **Before** *StartSection*

`start_section=SectionBreak('StartSection', break_offset='Before'),`

In [7]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    #end_on_first_item=True,
    processor=sub_section
    )
pprint(full_section.read(GENERIC_TEST_TEXT2))

[['StartSection A',
  'MiddleSection A',
  'EndSection A',
  'Unwanted text between sections',
  'StartSection B',
  'MiddleSection B',
  'EndSection B',
  'StartSection C',
  'MiddleSection C',
  'EndSection C',
  'Even more text to be ignored']]


- ![Good](../examples/Valid.png) Includes all lines after first *StartSection* in single sub-list.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A',<br>
       'Unwanted text between sections',<br>
       'StartSection B',<br>
       'MiddleSection B',<br>
       'EndSection B',<br>
       'StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C',<br>
       'Even more text to be ignored']<br>
    ]</code></td>
  <td><code>
    [<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A',<br>
       'Unwanted text between sections',<br>
       'StartSection B',<br>
       'MiddleSection B',<br>
       'EndSection B',<br>
       'StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C',<br>
       'Even more text to be ignored']<br>
    ]</code></td>
</table>



### Add end to Section Definition

- Section start **Before** *StartSection*
- Section end **After** *EndSection*

```python
full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    processor=sub_section 
    )
```

In [8]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    #end_on_first_item=True,    
    processor=sub_section
    )

test_iter = BufferedIterator(GENERIC_TEST_TEXT2)
pprint(full_section.read(test_iter))
pprint(full_section.read(test_iter))
pprint(full_section.read(test_iter))
pprint(full_section.read(test_iter))

[['StartSection A', 'MiddleSection A', 'EndSection A']]
[['StartSection B', 'MiddleSection B', 'EndSection B']]
[['StartSection C', 'MiddleSection C', 'EndSection C']]
[]


- ![Good](../examples/Valid.png) Includes all three lines of first section in single sub-list.
- skips the *'Unwanted text between sections'* line.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A']<br>
    ]</code></td>
  <td><code>
    [<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A']<br>
    ]</code></td>
</tr>
<tr>
  <td><code>
    [<br>
      ['StartSection B',<br>
       'MiddleSection B',<br>
       'EndSection B']<br>
    ]</code></td>
  <td><code>
    [<br>
      ['StartSection B',<br>
       'MiddleSection B',<br>
       'EndSection B']<br>
    ]</code></td>
</tr>
<tr>
  <td><code>
    [<br>
      ['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C']<br>
    ]</code></td>
  <td><code>
    [<br>
      ['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C']<br>
    ]</code></td>
</tr>
<tr><td><code>[]</code></td><td><code>[]</code></td></tr>
</table>



### Define Muti-Section to Read Both Sections

Add end to Section Definition
- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- Simple subsection
- Multi Section defines Full Section as Sub Section with no start or end (All lines)

```python
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    processor=sub_section 
    )

multi_section = Section(
    section_name='Multi',
    processor=full_section 
    )
```

In [9]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    processor=sub_section,
    #end_on_first_item=True
    )

multi_section = Section(
    section_name='Multi',
    processor=full_section
    #end_on_first_item=True
    )

pprint(multi_section.read(GENERIC_TEST_TEXT2))

[[['StartSection A', 'MiddleSection A', 'EndSection A']],
 [['StartSection B', 'MiddleSection B', 'EndSection B']],
 [['StartSection C', 'MiddleSection C', 'EndSection C']]]


- ![Good](../examples/Valid.png)Includes all three lines of each section in its own sub-list.
- Skips the *'Unwanted text between sections'* line.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      [['StartSection A',<br>
        'MiddleSection A',<br>
        'EndSection A']],<br>
      [['StartSection Name: B',<br>
        'MiddleSection B',<br>
        'EndSection Name: B']]<br>
      [['StartSection Name: C',<br>
        'MiddleSection C',<br>
        'EndSection Name: C']]<br>
    ]</code></td>    
  <td><code>
    [<br>
      [['StartSection A',<br>
        'MiddleSection A',<br>
        'EndSection A']],<br>
      [['StartSection Name: B',<br>
        'MiddleSection B',<br>
        'EndSection Name: B']]<br>
      [['StartSection Name: C',<br>
        'MiddleSection C',<br>
        'EndSection Name: C']]<br>
    ]</code></td></tr>
</table>

### Set Same Start and End Breaks for Section

Add end to Section Definition
- Section start **Before** *StartSection*
- Section end **Before** *StartSection*
- Simple subsection
- Multi Section defines Full Section as Sub Section with no start or end (All lines)

```python
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    processor=sub_section 
    )

multi_section = Section(
    section_name='Multi',
    processor=full_section 
    )
```

In [10]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )
full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    processor=sub_section,
    #end_on_first_item=True
    )

multi_section = Section(section_name='Multi',
    processor=full_section,
    #end_on_first_item=True
    )

pprint(multi_section.read(GENERIC_TEST_TEXT2))

[[['StartSection A',
   'MiddleSection A',
   'EndSection A',
   'Unwanted text between sections']],
 [['StartSection B', 'MiddleSection B', 'EndSection B']],
 [['StartSection C',
   'MiddleSection C',
   'EndSection C',
   'Even more text to be ignored']]]


- ![Good](../examples/Valid.png) Includes *'Unwanted text between sections'* and
  *'Even more text to be ignored'*.
- Includes *'Unwanted text between sections'* because the end of section A is 
  triggered by *'StartSection B'*. 
- Includes *'Even more text to be ignored'* because there are no more 
  *'StartSection'* lines to trigger an `end_section` break.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      [['StartSection A',<br>
        'MiddleSection A',<br>
        'EndSection A',<br>
        'Unwanted text between sections']],<br>
      [['StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B']],<br>
      [['StartSection C',<br>
        'MiddleSection C',<br>
        'EndSection C',<br>
        'Even more text to be ignored']]<br>
    ]</code></td>
  <td><code>
    [<br>
      [['StartSection A',<br>
        'MiddleSection A',<br>
        'EndSection A',<br>
        'Unwanted text between sections']],<br>
      [['StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B']],<br>
      [['StartSection C',<br>
        'MiddleSection C',<br>
        'EndSection C',<br>
        'Even more text to be ignored']]<br>
    ]</code></td>
  </tr>
</table>

### Add `end_on_first_item=True` to Section

Add end to Section Definition
- Section start **Before** *StartSection*
- Section end **After** *StartSection*
- Simple subsection

```python
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    end_on_first_item=True,
    processor=sub_section 
    )
```

In [11]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )
full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    end_on_first_item=True,
    processor=sub_section
    )

test_iter = BufferedIterator(GENERIC_TEST_TEXT2)
pprint(full_section.read(test_iter))
pprint(full_section.read(test_iter))

test_iter = iter(GENERIC_TEST_TEXT2)
pprint(full_section.read(test_iter))
pprint(full_section.read(test_iter))

[]
[]
[]
[]


- ![Good](../examples/Valid.png) Results in empty list because it both starts 
  and ends on first *'StartSection'*.

|Expected|Actual|
|-|-|
|[]|[]|

### <span style="color: red; font-weight: Bold">Endless loop with `multi_section`</span>

<span style="color: red; font-weight: Bold; font-variant: small-caps">
Do Not Try This
</span>

- Use `multi_section` to read multiple empty sections
```python
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    end_on_first_item=True,
    processor=sub_section 
    )
multi_section = Section(section_name='Multi',
    processor=full_section
    )
print(multi_section.read(GENERIC_TEST_TEXT2))
```
- `full_section` both starts and ends on first *'StartSection'*.
- No progress is made in in iterating through the source.
- Trying to read multiple sections will result in an endless loop.

**********

## SubSection Break Options
- Section break checks start with the top most section
- The section level containing the break will affect the way the list are nested.

### Add `start_section` to *sub_section* and remove it from *full_section*

- SubSection start **Before** *StartSection*
- Section end **After** *EndSection*


```python
sub_section = Section(section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before')
    )

full_section = Section(section_name='Full',
    end_section=SectionBreak('EndSection', break_offset='After'),
    processor=sub_section 
    )

multi_section = Section(section_name='Multi',
    processor=full_section
    )
```

In [12]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )
full_section = Section(
    section_name='Full',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    processor=sub_section,
    #end_on_first_item=True
    )

multi_section = Section(section_name='Multi',
    processor=full_section,
    #end_on_first_item=True
    )

In [13]:
pprint(multi_section.read(GENERIC_TEST_TEXT2))

[[['StartSection A', 'MiddleSection A', 'EndSection A']],
 [['StartSection B', 'MiddleSection B', 'EndSection B']],
 [['StartSection C', 'MiddleSection C', 'EndSection C']]]


- ![Good](../examples/Valid.png) behaves the same way as when `start_section` 
 was defined in *full_section*.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      [['StartSection A',<br>
        'MiddleSection A',<br>
        'EndSection A']],<br>
      [['StartSection Name: B',<br>
        'MiddleSection B',<br>
        'EndSection Name: B']]<br>
      [['StartSection Name: C',<br>
        'MiddleSection C',<br>
        'EndSection Name: C']]<br>
    ]</code></td>    
  <td><code>
    [<br>
      [['StartSection A',<br>
        'MiddleSection A',<br>
        'EndSection A']],<br>
      [['StartSection Name: B',<br>
        'MiddleSection B',<br>
        'EndSection Name: B']]<br>
      [['StartSection Name: C',<br>
        'MiddleSection C',<br>
        'EndSection Name: C']]<br>
    ]</code></td></tr>
</table>

### Add `end_section` to *sub_section* and remove it from *full_section*

- SubSection end **After** *EndSection*
- Section start **Before** *StartSection*

```python
sub_section = Section(section_name='SubSection',
    end_section=SectionBreak('EndSection', break_offset='After'),
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before')
    processor=sub_section 
    )

multi_section = Section(section_name='Multi',
    processor=full_section
    )
```

In [14]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )
full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    processor=sub_section,
    #end_on_first_item=True
    )

multi_section = Section(section_name='Multi',
    processor=full_section,
    #end_on_first_item=True
    )

pprint(multi_section.read(GENERIC_TEST_TEXT2))

[[['StartSection A', 'MiddleSection A', 'EndSection A'],
  ['Unwanted text between sections',
   'StartSection B',
   'MiddleSection B',
   'EndSection B'],
  ['StartSection C', 'MiddleSection C', 'EndSection C'],
  ['Even more text to be ignored']]]


- ![Good](../examples/Valid.png) Includes *'Unwanted text between sections'* and
  *'Even more text to be ignored'*.
- `sub_section` has no `start_section` defined, so the it starts immediately. 
- `full_section` never ends and so the second sub-section starts right after 
  the first one ends. 
- **Note:** 
    - sub-list nesting is different:
        > `[[[Section Lines], [Next Section Lines], ... ]]`<br>
        > VS.<br>
        > `[[[Section Lines]], [[Next Section Lines]], ... ]`<br>
    - *'Even more text to be ignored'* is in its own sub-section.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [[<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A'],<br>
      ['Unwanted text between sections',<br>
       'StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B'],<br>
      ['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C'],<br>
      ['Even more text to be ignored']<br>
    ]]</code></td>
  <td><code>
    [[<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A'],<br>
      ['Unwanted text between sections',<br>
       'StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B'],<br>
      ['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C'],<br>
      ['Even more text to be ignored']<br>
    ]]</code></td>
  </tr>
</table>

### Add both `start_section` and `end_section` to *sub_section*. No breaks defined in *full_section*

- SubSection start **Before** *StartSection*
- SubSection end **After** *EndSection*


```python
sub_section = Section(section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'
    end_section=SectionBreak('EndSection', break_offset='After'),
    )

full_section = Section(section_name='Full',
    processor=sub_section 
    )

multi_section = Section(section_name='Multi',
    processor=full_section
    )
```

In [15]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )
full_section = Section(
    section_name='Full',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    processor=sub_section,
    #end_on_first_item=True
    )

multi_section = Section(section_name='Multi',
    processor=full_section,
    #end_on_first_item=True
    )

pprint(multi_section.read(GENERIC_TEST_TEXT2))

[[['StartSection A', 'MiddleSection A', 'EndSection A'],
  ['StartSection B', 'MiddleSection B', 'EndSection B'],
  ['StartSection C', 'MiddleSection C', 'EndSection C']]]


- ![Good](../examples/Valid.png) similar results to having both breaks defined in *full_section*.
- **Note:** sub-list nesting is different:
    > `[[[Section Lines], [Next Section Lines], ... ]]`<br>
    > VS.<br>
    > `[[[Section Lines]], [[Next Section Lines]], ... ]`<br>

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [[<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A'],<br>
      ['StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B'],<br>
      ['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C']<br>
    ]]</code></td>
  <td><code>
    [[<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A'],<br>
      ['StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B'],<br>
      ['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C']<br>
    ]]</code></td>
  </tr>
</table>

### Read from *full_section* instead of *multi_section*

In [16]:
pprint(full_section.read(GENERIC_TEST_TEXT2))

[['StartSection A', 'MiddleSection A', 'EndSection A'],
 ['StartSection B', 'MiddleSection B', 'EndSection B'],
 ['StartSection C', 'MiddleSection C', 'EndSection C']]


- ![Good](../examples/Valid.png) Similar results to reading *multi_section*.

- **Note:** One less list level.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A'],<br>
      ['StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B'],<br>
      ['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C']<br>
    ]</code></td>
  <td><code>
    [<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A'],<br>
      ['StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B'],<br>
      ['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C']<br>
    ]</code></td>
  </tr>
</table>

### Add `start_section` and `end_section` to both *sub_section* and *full_section*.

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection start **Before** *StartSection*
- SubSection end **After** *EndSection*

```python
sub_section = Section(section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'
    end_section=SectionBreak('EndSection', break_offset='After'),
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'
    end_section=SectionBreak('EndSection', break_offset='After'),
    processor=sub_section 
    )

multi_section = Section(section_name='Multi',
    processor=full_section
    )
```

In [17]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )
full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    processor=sub_section,
    #end_on_first_item=True
    )

multi_section = Section(section_name='Multi',
    processor=full_section,
    #end_on_first_item=True
    )

pprint(multi_section.read(GENERIC_TEST_TEXT2))

[[['StartSection A', 'MiddleSection A', 'EndSection A']],
 [['StartSection B', 'MiddleSection B', 'EndSection B']],
 [['StartSection C', 'MiddleSection C', 'EndSection C']]]


- ![Good](../examples/Valid.png) behaves the same way as when *sub_section* 
  did not have any breaks defined (including list levels).

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      [['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A']],<br>
      [['StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B']],<br>
      [['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C']]<br>
    ]</code></td>
  <td><code>
    [<br>
      [['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A']],<br>
      [['StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B']],<br>
      [['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C']]<br>
    ]</code></td>
  </tr>
</table>

### Change Start Sub-Section to **After**

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection start **After** *StartSection*

```python
sub_section = Section(section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='After')
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    processor=[sub_section] 
    )
```

In [18]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='After')
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    processor=sub_section 
    )

multi_section = Section(section_name='Multi',
    processor=full_section,
    #end_on_first_item=True
    )

pprint(multi_section.read(GENERIC_TEST_TEXT2))

[[['MiddleSection A', 'EndSection A']],
 [['MiddleSection B', 'EndSection B']],
 [['MiddleSection C', 'EndSection C']]]


- ![Good](../examples/Valid.png) *StartSection* lines are dropped.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      [['MiddleSection A',
       'EndSection A']],<br>
      [['MiddleSection B',
        'EndSection B']],<br>
      [['MiddleSection C',
       'EndSection C']]<br>
    ]</code></td>
  <td><code>
    [<br>
      [['MiddleSection A',
       'EndSection A']],<br>
      [['MiddleSection B',
        'EndSection B']],<br>
      [['MiddleSection C',
       'EndSection C']]<br>
    ]</code></td>
  </tr>
</table>

### Run with two-line sections:
> `[`<br>
> `'Text to be ignored',`<br>
> `'StartSection Name: A',`<br>
> `'EndSection Name: A',`<br>
> `'Text between sections',`<br>
> `'StartSection Name: B',`<br>
> `'EndSection Name: B',`<br>
> `'More text to be ignored'`<br>
> `]`<br>

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection start **After** *StartSection*

```python
sub_section = Section(section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='After')
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    processor=[sub_section] 
    )
```

In [20]:
GENERIC_TEST_TEXT = [
    'Text to be ignored',
    'StartSection Name: A',
    'EndSection Name: A',
    'Text between sections',
    'StartSection Name: B',
    'EndSection Name: B',
    'More text to be ignored',
    ]

In [21]:
pprint(multi_section.read(GENERIC_TEST_TEXT))

[[['EndSection Name: A']], [['EndSection Name: B']]]


- ![Good](../examples/Valid.png) Results in both 'EndSections'
<table><thead><th>Expected</th><th>Actual</th></thead>
 <tr> <td><code>
    [<br>
      [['EndSection Name: A']],<br>
      [['EndSection Name: B']]<br>
    ]</code></td>
  <td><code>
    [<br>
      [['EndSection Name: A']],<br>
      [['EndSection Name: B']]<br>
    ]</code></td></tr>
</table>

In [22]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before')
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    processor=sub_section 
    )

multi_section = Section(section_name='Multi',
    processor=full_section,
    #end_on_first_item=True
    )

pprint(multi_section.read(GENERIC_TEST_TEXT2))

[[['EndSection A']], [['EndSection B']], [['EndSection C']]]


- ![Good](../examples/Valid.png) Results in three 'EndSections'
<table><thead><th>Expected</th><th>Actual</th></thead>
 <tr> <td><code>
    [<br>
      [['EndSection Name: A']],<br>
      [['EndSection Name: B']]<br>
      [['EndSection Name: C']]<br>
    ]</code></td>
  <td><code>
    [<br>
      [['EndSection Name: A']],<br>
      [['EndSection Name: B']]<br>
      [['EndSection Name: C']]<br>
    ]</code></td></tr>
</table>

In [23]:
GENERIC_TEST_TEXT

['Text to be ignored',
 'StartSection Name: A',
 'EndSection Name: A',
 'Text between sections',
 'StartSection Name: B',
 'EndSection Name: B',
 'More text to be ignored']

In [24]:
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A']]


- ![Good](../examples/Valid.png) Results in first 'EndSection'
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>[['EndSection Name: A']]</code></td>
<td><code>[['EndSection Name: A']]</code></td></tr>
</table>

### Add start to SubSection Definition
> `start_section=SectionBreak('EndSection', break_offset='Before')`

In [25]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    processor=[sub_section] 
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A',
  'Text between sections',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Good](../examples/Valid.png) Results in everything starting with first 'EndSection'.
- There is no `end_section`, so everything after `start_section` triggers will be read.
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>
[['EndSection Name: A',<br>
  'Text between sections',<br>
  'StartSection Name: B',
  'EndSection Name: B',<br>
  'More text to be ignored']]</code></td>
<td><code>
[['EndSection Name: A',<br>
  'Text between sections',<br>
  'StartSection Name: B',
  'EndSection Name: B',<br>
  'More text to be ignored']]
  </code></td></tr>
</table>

### Add *Start After* to Section Definition
> `start_section=SectionBreak('StartSection', break_offset='After')`

In [26]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    processor=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A',
  'Text between sections',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Good](../examples/Valid.png) Results in one line section starting with first 'EndSection'.
- There is no `end_section`, so everything after `start_section` triggers will be read.
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>
[['EndSection Name: A',<BR>
  'StartSection Name: B',
  'EndSection Name: B',<BR>
  'More text to be ignored']]
  </code></td>
<td><code>
[['EndSection Name: A',<BR>
  'StartSection Name: B',
  'EndSection Name: B',<BR>
  'More text to be ignored']]
  </code></td></tr>
</table>

### Add *Start After StartSection* to Section Definition and <br>*Start Before End Section* to SubSection Definition
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After')`
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before')`
    

In [27]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    processor=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A',
  'Text between sections',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Good](../examples/Valid.png) Two competing `start_section`s Results in one line section starting with first 'EndSection'.
- `full_section`'s `start_section=SectionBreak('StartSection', break_offset='After')` drives the start.
- There is no `end_section`, so everything after `start_section` triggers will be read.
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>
[['EndSection Name: A',<br>
  'Text between sections',<br>
  'StartSection Name: B',
  'EndSection Name: B',<br>
  'More text to be ignored']]</code></td>
<td><code>
  [['EndSection Name: A',<br>
  'Text between sections',<br>
  'StartSection Name: B',
  'EndSection Name: B',<br>
  'More text to be ignored']]</code></td></tr>
</table>

### Add *Start After StartSection* to Section Definition and <br>*Start Before End Section* to SubSection Definition and <br>set *End On First Line* for SubSection
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After')`
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`
> `end_on_first_item=True,`
    

In [28]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    processor=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A',
  'Text between sections',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Good](../examples/Valid.png) `end_on_first_item=True,` has no effect because no end_section is given.
- Two competing `start_section`s Results in one line section starting with first 'EndSection'.
- `full_section`'s `start_section=SectionBreak('StartSection', break_offset='After')` drives the start.
- There is no `end_section`, so everything after `start_section` triggers will be read.
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>
[['EndSection Name: A',<br>
  'Text between sections',<br>
  'StartSection Name: B',
  'EndSection Name: B',<br>
  'More text to be ignored']]</code></td>
<td><code>
  [['EndSection Name: A',<br>
  'Text between sections',<br>
  'StartSection Name: B',
  'EndSection Name: B',<br>
  'More text to be ignored']]</code></td></tr>
</table>

### Add *Start After StartSection* to Section Definition, and <br>for SubSection Definition, set *Start* to  *Before EndSection*<br> and *End* to *`True` (Always Break)*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After')`
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`
> `end_section=SectionBreak(True),`
    

In [29]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    processor=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A'], ['EndSection Name: B']]


- ![Good](../examples/Valid.png) Results in two one-line sections.
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>
[<br>
['EndSection Name: A'],<br>
['EndSection Name: B']<br>
]
  </code></td>
<td><code>
[<br>
['EndSection Name: A'],<br>
['EndSection Name: B']<br>
]
  </code></td></tr>
</table>

### Add *Start After StartSection* and <br>*End Before EndSection* to Section Definition, and <br>for SubSection Definition, set <br>*Start* to  *Before EndSection* and <br>*End* to *`True` (Always Break)*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After'),`<br>
> `end_section=SectionBreak('EndSection', break_offset='After'),`<br>
    
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`<br>
> `end_section=SectionBreak(True),`
    

In [30]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    processor=[sub_section] 
    )

pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A'], ['EndSection Name: B']]


- ![Good](../examples/Valid.png) Results in two one-line sections.
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>
[<br>
['EndSection Name: A'],<br>
['EndSection Name: B']<br>
]
  </code></td>
<td><code>
[<br>
['EndSection Name: A'],<br>
['EndSection Name: B']<br>
]
  </code></td></tr>
</table>

<b>Break Analysis</b>
<table><thead>
<th>Text Line</th><th>Full Section</th><th>SubSection</th>
</thead>
<tr><td>'Text to be ignored'</td><td>Skipped</td><td>Skipped</td></tr>
<tr><td>'StartSection Name: A'</td><td>Start Section After</td><td>Skipped</td></tr>
<tr><td><b>'EndSection Name: A'</b></td><td><u>Processed</u><br>No End Section<br>(<code>end_on_first_item=False</code>)</td><td>Start Section Before<br>(<u>Processed</u>)<br>No End Section<br>(<code>end_on_first_item=False</code>)</td></tr>
<tr><td>'Text between sections'</td><td><i>Processed</i></td><td>End Section Before</td></tr>
<tr><td>'StartSection Name: B'</td><td><i>Processed</i></td><td>Skipped</td></tr>
<tr><td><b>'EndSection Name: B'</b></td><td><u>Processed</u><br>End Section After</td><td>Start Section Before<br>(<u>Processed</u>)<br>No End Section<br>(<code>end_on_first_item=False</code>)</td></tr>
<tr><td>'More text to be ignored'</td><td>End of Section</td><td>Not Reached</td></tr>
 </table>

### Add *Start __Before__ StartSection* and <br>*End __Before__ EndSection* to Section definition, and <br>for *SubSection* definition, set <br>*Start* to  *__Before__ EndSection* and <br>*End* to *`True` (Always Break)*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='Before'),`<br>
> `end_section=SectionBreak('EndSection', break_offset='After'),`
    
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`<br>
> `end_section=SectionBreak(True),`
    

In [31]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    processor=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A']]


- ![Good](../examples/Valid.png) Results in single line section.
- 'EndSection Name: A' is now the second line in Full Section, so the section ends after this line.
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>[['EndSection Name: A']]</code></td>
<td><code>[['EndSection Name: A']]</code></td></tr>
</table>

### Add *Start __Before__ StartSection* and <br>*End Before EndSection* to Section Definition, and <br>for SubSection Definition, set <br>*Start* to *__Before__ EndSection*, <br>*End* to *`True` (Always Break)* and <br>*Keep Partial* to *`True`*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='Before'),`<br>
> `end_section=SectionBreak('EndSection', break_offset='After'),` <br>   
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`<br>
> `end_section=SectionBreak(True),`<br>
> `keep_partial=True,`
    

In [32]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    processor=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A']]


- ![Good](../examples/Valid.png) Results in the same single line section.
- `keep_partial=True` has no effect.
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>[['EndSection Name: A']]</code></td>
<td><code>[['EndSection Name: A']]</code></td></tr>
</table>

### Add *Start __Before__ StartSection* and <br>*End Before EndSection* to Section Definition, and <br>don't set any SectionBreaks for SubSection Definition, 
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='Before'),`<br>
> `end_section=SectionBreak('EndSection', break_offset='After'),`    
    

In [33]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    processor=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A']]


- ![Good](../examples/Valid.png) Results in Two line section
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>[[<br>'StartSection Name: A',<br>'EndSection Name: A'<br>]]</code></td>
<td><code>[[<br>'StartSection Name: A',<br>'EndSection Name: A'<br>]]</code></td></tr>
</table>

### Add *Start __After__ StartSection* and <br>*End Before EndSection* to Section Definition, and <br>don't set any SectionBreaks for SubSection Definition, 
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After'),`<br> 
> `end_section=SectionBreak('EndSection', break_offset='After'),` <br>    
    

In [34]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    processor=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A',
  'Text between sections',
  'StartSection Name: B',
  'EndSection Name: B']]


- ![Good](../examples/Valid.png) Starts with first 'EndSection' ends with second 'EndSection'.
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>
[['EndSection Name: A',<br>
  'Text between sections',<br>
  'StartSection Name: B',<br>
  'EndSection Name: B']]
</code></td>
<td><code>
[['EndSection Name: A',<br>
  'Text between sections',<br>
  'StartSection Name: B',<br>
  'EndSection Name: B']]
</code></td></tr>
</table>

### Set `end_on_first` to *True*. <br>*Start __After__ StartSection* and <br>*End Before EndSection* to Section Definition, and <br>don't set any SectionBreaks for SubSection Definition, 
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After'),`<br> 
> `end_section=SectionBreak('EndSection', break_offset='After'),`<br>
> `end_on_first_item=True`<br>

In [35]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    end_on_first_item=True,
    #subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

['EndSection Name: A']


- ![Good](../examples/Valid.png) Results in one line section
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>[['EndSection Name: A']]</code></td>
<td><code>[['EndSection Name: A']]</code></td></tr>
</table>

## Three line sections

In [36]:
GENERIC_TEST_TEXT3 = [
    'Text to be ignored',
    'StartSection A',
    'MiddleSection A',
    'EndSection A',
    'Unwanted text between sections',
    'StartSection B',
    'Random text in the middle of a section',
    'MiddleSection B',
    'EndSection B',
    'StartSection C',
    'MiddleSection C',
    'EndSection C',
    'Even more text to be ignored', 
    ]

#### Single line subsection definitions 

In [37]:
start_sub_section = Section(
    section_name='StartSubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )

middle_sub_section = Section(
    section_name='MiddleSubSection',
    start_section=SectionBreak('MiddleSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )

end_sub_section = Section(
    section_name='EndSubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )

#### Defining ***three_part_section*** 
- Contains an ending break:
    > `end_section=SectionBreak('ignored', break_offset='Before')`.

- Contains 3 subsections:
    > `[start_sub_section, middle_sub_section, end_sub_section]`

In [38]:
three_part_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('ignored', break_offset='Before'),
    processor=[start_sub_section, middle_sub_section, end_sub_section]
    )
pprint(three_part_section.read(GENERIC_TEST_TEXT3))

BufferedIteratorValueError: Can't step back 9 items.
	only have 4 previous items available.

![Good](../examples/Valid.png) All 3 sections and subsections are completed. 

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          [<br>
            [
              ['StartSection Name: A'],<br> 
              ['MiddleSection A'],<br> 
              ['EndSection Name: A']
            ],<br>
            [
              ['StartSection Name: B'],<br> 
              ['MiddleSection A'],<br> 
              ['EndSection Name: B']
            ]<br>
            [
              ['StartSection Name: C'],<br> 
              ['MiddleSection C'],<br> 
              ['EndSection Name: C']
            ]<br>
          ]
        </code></td>
        <td><code>
          [<br>
            [
              ['StartSection Name: A'],<br> 
              ['MiddleSection A'],<br> 
              ['EndSection Name: A']
            ],<br>
            [
              ['StartSection Name: B'],<br> 
              ['MiddleSection A'],<br> 
              ['EndSection Name: B']
            ]<br>
            [
              ['StartSection Name: C'],<br> 
              ['MiddleSection C'],<br> 
              ['EndSection Name: C']
            ]<br>
          ]
        </code></td></tr>
</table>

- Section start **Before** *StartSection*
- Section end **After** *EndSection*

```python
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    processor=[sub_section] 
    )
```

In [ ]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='After')
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    processor=[sub_section] 
    )

pprint(full_section.read(GENERIC_TEST_TEXT3))

![Good](../examples/Valid.png) First section and all 3 subsections are completed. 

<table>
  <thead><th>Expected</th><th>Actual</th></thead>
  <tr><td><code>
    [[<br>
    'StartSection Name: A',<br> 
    'MiddleSection A',<br> 
    'EndSection Name: A'<br>
    ]]
    </code></td>
    <td><code>
    [[<br>
    'StartSection Name: A',<br> 
    'MiddleSection A',<br> 
    'EndSection Name: A'<br>
    ]]
  </code></td></tr>
</table>